In [9]:
import re
money = "459,200元"
match = re.search('[\d,]+', money)
print match.group()

459,200


In [22]:
sub_money = re.sub(",","",money)
print sub_money

459200元


In [10]:
money_list = money.split(',')

In [11]:
money_str = "".join(money_list)

In [12]:
money_list

['459', '200\xe5\x85\x83']

In [18]:
print money_str

459200元


## selenium:用chrome操作Agoda

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

URL="https://www.agoda.com/zh-tw/pages/agoda/default/DestinationSearchResult.aspx?asq=d55U80GmwPHP3MLniek%2b%2f1yBY%2fw5OEu%2fG1a41X%2f1OPzEkjwlidnWS%2fnIeW7UJJYUHkrrCUc3640lpdA0xiwYS44B6WsMbKG6n9PWyZrUBYZANK8hUx%2bLtOo10vXIEdnsBb%2bZ58XrzTZovriM0y21plSo6nV%2b%2fsWhzat7hBfJ1pYxfoPrkU1bRmIhfPCLM5HX&city=5085&tick=636136936073&pagetypeid=1&origin=TW&cid=-1&htmlLanguage=zh-tw&checkIn=2016-11-28&checkOut=2016-11-30&los=2&rooms=1&adults=2&children=0&isFromSearchBox=true&ckuid=b0b1f7ba-51ed-48b8-83a1-637d4a9bd984"

driver = webdriver.Remote("http://localhost:9515", webdriver.DesiredCapabilities.CHROME)
try:
    driver.get(URL)
    with open('tokyo_hotel.html', 'w') as f:
        f.write(driver.page_source.encode('utf-8'))

    for x in range(1,25):
        element = WebDriverWait(driver,15).until(EC.presence_of_element_located((By.ID,"paginationNext")))
        driver.execute_script("return arguments[0].scrollIntoView()", element)
        time.sleep(3)
        with open('tokyo_hotel_{}.html'.format(x), 'w') as f:
            f.write(driver.page_source.encode('utf-8'))
        element.click()
finally:
    driver.close()

ElementNotVisibleException: Message: element not visible
  (Session info: chrome=54.0.2840.87)
  (Driver info: chromedriver=2.25.426923 (0390b88869384d6eb0d5d09729679f934aab9eed),platform=Windows NT 6.1.7601 SP1 x86_64)


## 擷取Agoda Tokyo 酒店資料

In [68]:
import json
                
from bs4 import BeautifulSoup
FILE = "tokyo_hotel_1.html"
file = open('detail3.json', 'w')
detaillist=[]

for page in range(1,25):
    with open(FILE, 'r') as f:
        html_file = f.read()
        soup = BeautifulSoup(html_file, 'lxml')
        details = soup.select('section.hotel-item-box')
        for detail in details:
#             print detail.select_one('h3').text.strip()
#             print detail.select_one('span.areacity-name').text.strip()
#             print detail.select_one('span.price').text
#             print detail.select_one('span.soft-red').text
            detail_dict = {}
            detail_dict['1_hotel_name'] = detail.select_one('h3').text.strip()
            detail_dict['2_area_name'] = detail.select_one('span.areacity-name').text.strip()
            detail_dict['3_price'] = detail.select_one('span.price').text
            detail_dict['4_price_dicounted'] = detail.select_one('span.soft-red').text
            detaillist.append(detail_dict)
            print detail_dict
        

{'2_area_name': u'\u65b0\u5bbf, \u6771\u4eac', '1_hotel_name': u'\u683c\u62c9\u65af\u9e97\u65b0\u5bbf\u9152\u5e97 (Hotel Gracery Shinjuku)', '3_price': u'6,668', '4_price_dicounted': u'4,982'}
{'2_area_name': u'\u4e0a\u91ce, \u6771\u4eac', '1_hotel_name': u'\u4e09\u4e95\u82b1\u5712\u98ef\u5e97\u4e0a\u91ce (Mitsui Garden Hotel Ueno)', '3_price': u'8,962', '4_price_dicounted': u'4,107'}
{'2_area_name': u'\u4e0a\u91ce, \u6771\u4eac', '1_hotel_name': u'\u6771\u4eac\u4e0a\u91ce\u98ef\u5e97 (Tokyo UenoHotel)', '3_price': u'4,634', '4_price_dicounted': u'2,381'}
{'2_area_name': u'\u65b0\u5bbf, \u6771\u4eac', '1_hotel_name': u'\u65b0\u5bbf\u71e6\u8def\u90fd\u5ee3\u5834\u5927\u98ef\u5e97 (Hotel Sunroute Plaza Shinjuku)', '3_price': u'5,652', '4_price_dicounted': u'4,050'}
{'2_area_name': u'\u5fa1\u53f0\u5834, \u6771\u4eac', '1_hotel_name': u'\u6771\u4eac\u53f0\u5834\u65e5\u822a\u5927\u9152\u5e97 (Grand Nikko Tokyo Daiba)', '3_price': u'13,341', '4_price_dicounted': u'4,158'}
{'2_area_name': u'\

# to json file

In [69]:
import json
hoteljson = json.dumps(detaillist, ensure_ascii=False)
with open('detail5.json', 'w') as a:
    a.write(hoteljson.encode('utf-8'))

## 撈政府採購網資料

In [36]:
def str_to_dict(mystr):
    res_dict = {}
    for line in raw_form.split("\n"):
        if len(line) != 0:
    #         print line
            k = line.split(":")[0]
            v = line.split(":")[1]
            res_dict[k] = v
    return res_dict

In [58]:
import json
import requests
from bs4 import BeautifulSoup

URL = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=basic"

raw_form = '''
method:search
searchMethod:true
tenderUpdate:
searchTarget:
orgName:
orgId:
hid_1:1
tenderName:
tenderId:
tenderType:tenderDeclaration
tenderWay:1,2,3,4,5,6,7,10,12
tenderDateRadio:on
tenderStartDateStr:105/10/28
tenderEndDateStr:105/11/03
tenderStartDate:105/11/03
tenderEndDate:105/11/03
isSpdt:N
proctrgCate:
btnQuery:查詢
hadUpdated:
'''
govorder = []
data = str_to_dict(raw_form)
r = requests.Session()
res = r.post(URL, data=data)
soup = BeautifulSoup(res.text, 'lxml')

rows = soup.select_one('#print_area').select('tr')
for idx, row in enumerate(rows):
    if idx != 0 and idx != 101:
        cols = row.select('td')
        rows_dict = {}
        rows_dict['serial_no'] = cols[0].text.strip()
        rows_dict['org_name'] = cols[1].text.strip()
        rows_dict['pj_name'] = cols[2].text.strip().split()[1]
        rows_dict['pj_way'] = cols[4].text.strip()
        rows_dict['deadline'] = cols[7].text
# for k in rows_dict:
#             print rows_dict[k]
        govorder.append(rows_dict)
        print rows_dict


{'org_name': u'\u5c4f\u6771\u7e23\u9ad8\u6a39\u9109\u820a\u5bee\u570b\u6c11\u5c0f\u5b78', 'pj_way': u'\u516c\u958b\u53d6\u5f97\u5831\u50f9\u55ae\u6216\u4f01\u5283\u66f8', 'serial_no': u'1', 'deadline': u'105/11/08\n', 'pj_name': u'\u820a\u5bee\u570b\u5c0f105\u5e74\u6c38\u7e8c\u6821\u5712\u5c40\u90e8\u6539\u9020\u8a08\u756b\u6c34\u96fb\u5de5\u7a0b'}
{'org_name': u'\u81fa\u7063\u83f8\u9152\u80a1\u4efd\u6709\u9650\u516c\u53f8\u8c50\u539f\u6372\u83f8\u7814\u767c\u88fd\u9020\u5de5\u5ee0', 'pj_way': u'\u516c\u958b\u62db\u6a19', 'serial_no': u'2', 'deadline': u'105/11/17\n', 'pj_name': u'\u6c70\u63db\u6372\u83f8\u6a5f\u96f7\u5c04\u6253\u5b54\u8a2d\u5099\u6a19\u6848'}
{'org_name': u'\u81fa\u7063\u83f8\u9152\u80a1\u4efd\u6709\u9650\u516c\u53f8\u5357\u6295\u9152\u5ee0', 'pj_way': u'\u516c\u958b\u62db\u6a19', 'serial_no': u'3', 'deadline': u'105/11/15\n', 'pj_name': u'0.7L+0.2L\u7389\u5c0a\u9ea5\u82bd\u5a01\u58eb\u5fcc\u79ae\u76d2\u7d44'}
{'org_name': u'\u53f0\u7063\u4e2d\u6cb9\u80a1\u4efd\u6709\

In [59]:
# load進json
import json
govjson = json.dumps(govorder, ensure_ascii=False)
with open('govorder.json', 'w') as a:
    a.write(govjson.encode('utf-8'))